# 1절 모델 하이퍼파라미터 튜닝 이해하기
지금까지 머신러닝 모델의 성능을 높이기 위해서 학습 데이터 전처리와 정제, 데이터의 특징 선택과 추출, 그리고 여러 가지 머신러닝 모델을 테스트해 보았습니다. 이번에는 세부 튜닝 방법으로 하이퍼파라미터 조정을 통해 모델의 성능을 개선해 봅니다.<br> 

하이퍼파라미터는 모델을 훈련시킬 때 사용자가 직접 설정하는 값입니다. 학습률(learning rate), 의사결정나무의 최대 깊이(max depth) 값 등 사용자가 직접 설정해야 하는 파라미터 값이 상당히 많습니다. 전통적으로 하이퍼파라미터 튜닝은 수동으로 수행되었습니다. 해당 하이퍼파라미터 사용 사례와 관련 도메인 경험이 있는 사람이 경험과 직관에 따라 하이퍼파라미터를 수동으로 설정해서 모델을 훈련하고 검증하여 좋은 성능이 나올 때까지 반복했습니다. <br>
수동으로 하이퍼파라미터를 튜닝하는 것은 노동 집약적이고 효율적인 방법이 아닙니다. 현재 하이퍼파라미터를 자동으로 튜닝하는 대표적인 기법으로는 그리드 서치(Grid Search)와 랜덤 서치(Random Search)가 있습니다. 사이킷런 라이브러리의 model_selection 서브패키지에 GridSearchCV, RandomizedSearchCV와 RandomizedSearchCV로 머신러닝 모델의 하이퍼파라미터를 튜닝하고 모델 성능을 개선해 봅니다. 

## 1. 그리드 서치
### 1) 그리드 서치 이해하기
그리드 서치(Grid Search)는 가능한 모든 하이퍼파라미터 값의 조합에 대해 모델 성능을 측정하고 비교하면서 최적의 하이퍼파라미터 값을 찾는 방식입니다. 모든 값을 탐색한다는 점에서 철저한 방식인 반면 시간이 많이 소요되는 비효율적인 방법일 수도 있습니다. 

### 2) 그리드 서치 실습하기
사이킷런 라이브러리에서 제공하는 GridSearchCV를 사용하는 방법은 간단합니다. 사용자가 모델의 하이퍼파라미터 값을 리스트로 GridSearchCV에 입력하면, 리스트에 있는 값들의 모든 조합에 대해 모델 성능을 평가하여 최적의 조합을 찾습니다. 먼저 튜닝하려는 하이퍼파라미터의 값을 리스트로 지정하고 딕셔너리로 자료형을 구성합니다. 그리고 sklearn.model_selection 모듈에 있는 GridSearchCV 클래스의 객체를 만들고, estimator 객체를 훈련하고 튜닝합니다.<br> 
예제 코드에서는 그리드 서치를 사용하여 랜덤 포레스트 분류기의 여러 파라미터를 튜닝합니다. 해당 코드를 통해 사용법을 참고로 확인한 후 실습해 봅니다.<br>
랜덤 포레스트 분류기의 하이퍼파라미터는 아래와 같습니다.<br> 


<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
<b>| 하이퍼파라미터 |</b><br>
* max_depth：랜덤 프로세트 모델에서 각 의사결정나무의 최대 깊이<br>
* max_features：랜덤 포레스트 모델이 각 분할에서 시도할 수 있는 최대 특성 수<br>
* n_estimators：의사결정나무 수<br>
* min_samples_leaf：각 트리의 리프 노드에 있어야 하는 최소 샘플 수<br>
* min_samples_split：각 트리의 내부 노드를 분할하는 데 필요한 최소 샘플 수
</div>

GridSearchCV클래스의 객체 생성 인자는 다음과 같습니다. 
* estimator：estimator 객체
* param_grid：튜닝하려는 하이퍼파라미터의 딕셔너리를 지정
* scoring：모델의 성능을 평가하는 전략으로, 기본값은 ‘accuracy’
* n_jobs：병렬로 실행할 작업 수이며, -1은 모든 프로세서를 사용한다는 의미이고, 기본값은 1
* refit：가장 최적의 하이퍼파라미터를 찾은 후 입력된 estimator 객체를 해당 하이퍼파라미터로 재학습시키는 것으로, 기본값은 True
* cv：교차 검증을 위한 fold 횟수

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris  # 예시 데이터셋

# 예시 데이터 로드 (Iris 데이터셋 사용)
data = load_iris()
X = data.data
y = data.target

# 데이터셋을 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 그리드 서치를 위한 RandomForestClassifier 설정
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

estimator = RandomForestClassifier()

param_grid = {'max_depth': [3, 5, 10, None],
              'n_estimators': [10, 100, 200],
              'max_features': [1, 3, 5, 7],
              'min_samples_leaf': [1, 2, 3],
              'min_samples_split': [1, 2, 3]}

gs_cv = GridSearchCV(estimator=estimator,
                     param_grid=param_grid,
                     scoring='accuracy',
                     cv=3)

# 그리드 서치를 훈련 데이터에 적용
model = gs_cv.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 성능 출력
#print("Best Parameters:", gs_cv.best_params_)
#print("Best Score:", gs_cv.best_score_)


C:\Users\ubion\anaconda3\envs\aitensor\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
432 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
432 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ubion\anaconda3\envs\aitensor\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ubion\anaconda3\envs\aitensor\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "C:\Users\ubion\anaconda3\envs\aitensor\lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_cons

In [2]:
print(f'Best hyperparameters : {model.best_params_}')
print(f'Best score : {model.best_score_}')

Best hyperparameters : {'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 200}
Best score : 0.9666666666666667


In [3]:
clf=model.best_estimator_
clf.fit(X_train, y_train)
print(f'테스트 정확도 : {clf.score(X_train, y_train)}')

테스트 정확도 : 0.9666666666666667


## 2. 랜덤 서치
### 1) 랜덤 서치 이해하기
랜덤 서치(Random Search)는 그리드 서치와 유사하지만 가능한 각 하이퍼파라미터 조합에 대해 훈련하고 점수를 매기는 대신 랜덤 조합이 선택됩니다. 시간 및 리소스 제약에 따라 검색 반복 횟수를 설정할 수 있으며, 파라미터 탐색 범위가 넓거나 연속적인 값을 탐색해야 하는 경우에 랜덤 서치가 효율적입니다. 



### 2) 랜덤 서치 실습하기
튜닝하려는 매개변수들의 값을 리스트로 지정하고 딕셔너리로 변수를 만든 다음, sklearn.model_selection 모듈에 있는 RadnomizedSearchCV 클래스의 객체를 만들고, estimator 객체를 훈련하고 튜닝합니다. 예제 코드에서는 랜덤 서치를 사용하여 랜덤 포레스트 분류기의 여러 파라미터를 튜닝합니다. 해당 코드를 통해 사용법을 참고로 확인한 후 실습해 봅니다. 

> 하이퍼파라미터
> <div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
    * max_depth：랜덤 포레스트 모델에서 각 의사결정나무의 최대 깊이<br>
    * n_estimators：의사결정나무 수<br>
    * criterion：분할 품질을 측정하는 기능<br>
    * min_samples_leaf：각 트리의 리프 노드에 있어야 하는 최소 샘플 수<br>
    * min_samples_split：각 트리의 내부 노드를 분할하는 데 필요한 최소 샘플 수</div>

RandomizedSearchCV 클래스 객체 생성 인자는 다음과 같습니다.<br>
* estimator：estimator 객체<br>
* n_iter：파라미터 검색 횟수, 지정한 횟수만큼만 조합을 반복하여 평가, 기본값은 10<br>
* param_distributions：튜닝하려는 하이퍼파라미터의 딕셔너리를 지정<br>
* scoring：모델의 성능을 평가하는 전략으로, 기본값은 ‘accuracy’<br>
* n_jobs：병렬로 실행할 작업수이며, -1은 모든 프로세서를 사용한다는 의미이고, 기본값은 1<br>* refit：가장 최적의 하이퍼파라미터를 찾은 후 입력된 estimator 객체를 해당 하이퍼파라미터로 재학습시키는 것으로, 기본값은 True<br>
* cv：교차 검증을 위한 fold 횟수

In [6]:
import numpy as np
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# 모델 정의
estimator = RandomForestClassifier()

# 하이퍼파라미터 검색 범위 정의
param_distributions = {'max_depth': list(np.arange(3, 13, step=3)) + [None],
                       'n_estimators': np.arange(10, 320, step=100),
                       'max_features': randint(1, 7),
                       'criterion': ['gini', 'entropy'],
                       'min_samples_leaf': randint(1, 4),
                       'min_samples_split': np.arange(2, 8, step=2)}

# RandomizedSearchCV 정의
rs_cv = RandomizedSearchCV(estimator=estimator,
                           param_distributions=param_distributions,
                           n_iter=10,
                           scoring='accuracy',
                           n_jobs=-1,
                           refit=True,
                           cv=3)

# 모델 학습
model = rs_cv.fit(X_train, y_train)

훈련 데이터세트를 사용하여 랜덤 서치를 실행한 후, 모델의 최적의 하이퍼파라미터 값은 best_params_ 속성에서, 모델의 최고 점수는 best_score_ 속성에서 확인할 수 있습니다. 

In [7]:
print(f'Best hyperparameters : {model.best_params_}')
print(f'Best score : {model.best_score_}')

Best hyperparameters : {'criterion': 'entropy', 'max_depth': 6, 'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 110}
Best score : 0.9666666666666667


테스트 데이터세트로 모델 성능을 평가합니다. 

In [8]:
clf=model.best_estimator_
clf.fit(X_train, y_train)
print(f'테스트 정확도 : {clf.score(X_test, y_test)}')

테스트 정확도 : 1.0
